<a href="https://colab.research.google.com/github/OnePassio/CE888/blob/master/Project/Experiment_1_Normal_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Score Kaggle of this Experiment is : 0.57321

## Load Library

In [0]:
# Initial Library
# data analysis
import pandas as pd
import numpy as np
import random as rand

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# ignore warning message
import warnings
warnings.filterwarnings('ignore')

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 1.Load Data from Kaggle
Source Data: https://www.kaggle.com/c/microsoft-malware-prediction/

!kaggle competitions download -c microsoft-malware-prediction

In [0]:
# load data from kaggle
import os
os.environ['KAGGLE_USERNAME'] = "thomastran89" # username from the json file
os.environ['KAGGLE_KEY'] = "6d1907f735446c9c560bb4371074fecd" # key from the json file
!kaggle competitions download -c microsoft-malware-prediction # api copied from kaggle
# Unzip the data
!sample_submission.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip
# Check files after unzip
!ls

100% 672M/672M [00:02<00:00, 292MB/s]

 81% 109M/134M [00:00<00:00, 238MB/s] 
100% 134M/134M [00:00<00:00, 273MB/s]
 99% 758M/768M [00:03<00:00, 252MB/s]
100% 768M/768M [00:03<00:00, 225MB/s]
/bin/bash: sample_submission.csv.zip: command not found
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
sample_data		   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [0]:
# load partial data for test first
sample_size= 1000000 # data too heavy and can not load by Google Colab -> Test on 1M first rows

# Load Train Data
df_train = pd.read_csv("train.csv",skiprows=0, nrows=sample_size)

# Load Test Data
df_test= pd.read_csv("test.csv")#,skiprows=0, nrows=sample_size)

df_train.head()
#df_test.head()


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,...,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,...,HDD,299451.0,0,4096.0,Desktop,18.9,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,2668.0,91656.0,4.0,5.0,...,HDD,102385.0,0,4096.0,Notebook,13.9,1366.0,768.0,Mobile,NaN,1.000000e+00,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8.0,31,UNKNOWN,0,OFFLINE,Retail,NaN,0.0,NOT_SET,NaN,628.0,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,4909.0,317701.0,4.0,5.0,...,SSD,113907.0,0,4096.0,Desktop,21.5,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7.0,30,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,142.0,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,NaN,117.0,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,1443.0,275890.0,4.0,5.0,...,UNKNOWN,227116.0,0,4096.0,MiniTower,18.5,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,17.0,64,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,355.0,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417e

# 2. Preprocessing Data

## 2.1. Remove some unused features to build model of dataset
Some column field data have a unique ID each row or have too much distinct values(>50) should be remove, because it will make hard for build ML model

In [0]:
# Get All Accept Field from dataset (Ignore the field with too much string values) to build ML model
MAX_VALUE_ACCEPT=50
listAcceptField_Train=[]
for x in list(df_train):
    count=df_train[x].nunique()
    #print("{} has unique value is:{}".format(x,count))
    if(count<MAX_VALUE_ACCEPT):
        listAcceptField_Train.append(x)

listAcceptField_Test=listAcceptField_Train.copy()
listAcceptField_Test.remove('HasDetections')

print("Accept Fields from dataset to build Model is:")
print(listAcceptField_Train)
print(listAcceptField_Test)

Accept Fields from dataset to build Model is:
['ProductName', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'OrganizationIdentifier', 'Platform', 'Processor', 'OsVer', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode', 'SMode', 'SmartScreen', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName', 'Census_HasOpticalDiskDrive', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSWUAutoUpdateOptionsName', 'Census_IsPortableOperatingSystem', 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census_IsFlightingInternal', 'Census_IsFlightsDisabled', '

In [0]:
#cache
df_train_id=df_train['MachineIdentifier'].copy()
df_test_id=df_test['MachineIdentifier'].copy()
# Just keep acceptable features
df_train=df_train[listAcceptField_Train].copy();
df_test=df_test[listAcceptField_Test].copy();

df_test.head()

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,win8defender,0,7.0,0,1.0,1.0,1,18.0,windows10,x64,10.0.0.0,768,rs2,Home,1.0,0,NaN,NaN,NaN,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,NaN,SSD,0,Notebook,Mobile,NaN,amd64,rs2_release,Core,CORE,Reset,37.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,0.0,7.0
1,win8defender,0,7.0,0,1.0,1.0,1,NaN,windows10,x64,10.0.0.0,768,rs3,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,NaN,HDD,0,Notebook,Mobile,NaN,amd64,rs3_release_svc_escrow,Core,CORE,UUPUpgrade,7.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,1.0,12.0
2,win8defender,0,7.0,0,2.0,1.0,1,NaN,windows10,x64,10.0.0.0,768,rs1,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,NaN,SSD,0,Notebook,Mobile,NaN,amd64,rs1_release,CoreSingleLanguage,CORE_SINGLELANGUAGE,Other,35.0,UNKNOWN,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,1.0,11.0
3,win8defender,0,7.0,0,2.0,1.0,1,NaN,windows10,x64,10.0.0.0,768,rs3,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,NaN,HDD,0,Notebook,Mobile,li-i,amd64,rs3_release,CoreSingleLanguage,CORE_SINGLELANGUAGE,Upgrade,26.0,Notify,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,0.0,1,0.0,0.0,0,0,0.0,0.0,10.0
4,win8defender,0,7.0,0,1.0,1.0,1,18.0,windows10,x64,10.0.0.0,768,rs3,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,NaN,HDD,1,Notebook,Mobile,NaN,amd64,rs3_release,CoreSingleLanguage,CORE_SINGLELANGUAGE,Update,29.0,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,1.0,3.0


### 2.2. Fill NaN missing value

In [0]:
## missing values on train.csv
for i in df_train.columns:
    if df_train[i].dtype == 'object':
      df_train[i] = df_train[i].fillna(df_train[i].mode().iloc[0])
    if (df_train[i].dtype == 'int' or df_train[i].dtype == 'float'):
      df_train[i] = df_train[i].fillna(np.mean(df_train[i]))

## missing values on test.csv
for i in df_test.columns:
    if df_test[i].dtype == 'object':
      df_test[i] = df_test[i].fillna(df_test[i].mode().iloc[0])
    if (df_test[i].dtype == 'int' or df_test[i].dtype == 'float'):
      df_test[i] = df_test[i].fillna(np.mean(df_test[i]))
      
df_test.head()

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,win8defender,0,7.0,0,1.0,1.0,1,18.000000,windows10,x64,10.0.0.0,768,rs2,Home,1.0,0,on,0.003175,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,mid,SSD,0,Notebook,Mobile,lion,amd64,rs2_release,Core,CORE,Reset,37.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,OEM:DM,0.000011,0.0,Retail,0.000281,1,4.290840e-07,0.0,0,0,0.0,0.0,7.0
1,win8defender,0,7.0,0,1.0,1.0,1,24.952712,windows10,x64,10.0.0.0,768,rs3,Home,1.0,0,on,0.003175,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,mid,HDD,0,Notebook,Mobile,lion,amd64,rs3_release_svc_escrow,Core,CORE,UUPUpgrade,7.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,Retail,0.000011,0.0,Retail,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,12.0
2,win8defender,0,7.0,0,2.0,1.0,1,24.952712,windows10,x64,10.0.0.0,768,rs1,Home,1.0,0,on,0.003175,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,mid,SSD,0,Notebook,Mobile,lion,amd64,rs1_release,CoreSingleLanguage,CORE_SINGLELANGUAGE,Other,35.0,UNKNOWN,0,IS_GENUINE,OEM:DM,0.000011,0.0,Retail,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,11.0
3,win8defender,0,7.0,0,2.0,1.0,1,24.952712,windows10,x64,10.0.0.0,768,rs3,Home,1.0,0,on,0.003175,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,mid,HDD,0,Notebook,Mobile,li-i,amd64,rs3_release,CoreSingleLanguage,CORE_SINGLELANGUAGE,Upgrade,26.0,Notify,0,IS_GENUINE,OEM:DM,0.000011,0.0,Retail,0.000000,1,0.000000e+00,0.0,0,0,0.0,0.0,10.0
4,win8defender,0,7.0,0,1.0,1.0,1,18.000000,windows10,x64,10.0.0.0,768,rs3,Home,1.0,0,on,0.003175,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,mid,HDD,1,Notebook,Mobile,lion,amd64,rs3_release,CoreSingleLanguage,CORE_SINGLELANGUAGE,Update,29.0,UNKNOWN,0,IS_GENUINE,Retail,0.000011,0.0,Retail,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,3.0


## 2.3. label encoding

In [0]:
# encoder, alternative is convert to dummies
number = LabelEncoder()
for i in df_train.columns:
    if (df_train[i].dtype == 'object'):
      df_train[i] = number.fit_transform(df_train[i].astype('str'))
      df_train[i] = df_train[i].astype('object')

for i in df_test.columns:
    if (df_test[i].dtype == 'object'):
      df_test[i] = number.fit_transform(df_test[i].astype('str'))
      df_test[i] = df_test[i].astype('object')

df_test.head()

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,4,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,768,2,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,4.0,5.0,2,1,0,33,3,42,0,3,4,2,5,37.0,0,0,1,0,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,0.0,7.0
1,4,0,7.0,0,1.0,1.0,1,24.952712,0,1,0,768,3,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,4.0,5.0,2,0,0,33,3,42,0,6,4,2,6,7.0,0,0,1,2,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,12.0
2,4,0,7.0,0,2.0,1.0,1,24.952712,0,1,0,768,1,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,2.0,5.0,2,1,0,33,3,42,0,0,7,5,3,35.0,5,0,1,0,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,11.0
3,4,0,7.0,0,2.0,1.0,1,24.952712,0,1,0,768,3,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,2.0,5.0,2,0,0,33,3,37,0,5,7,5,8,26.0,3,0,1,0,0.000011,0.0,7,0.000000,1,0.000000e+00,0.0,0,0,0.0,0.0,10.0
4,4,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,768,3,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,4.0,5.0,2,0,1,33,3,42,0,5,7,5,7,29.0,5,0,1,2,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,3.0


## 2.4. creating a new feature origin


In [0]:
df_train['origin'] = 0
df_test['origin'] = 1
df_test.head()

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,origin
0,4,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,768,2,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,4.0,5.0,2,1,0,33,3,42,0,3,4,2,5,37.0,0,0,1,0,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,0.0,7.0,1
1,4,0,7.0,0,1.0,1.0,1,24.952712,0,1,0,768,3,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,4.0,5.0,2,0,0,33,3,42,0,6,4,2,6,7.0,0,0,1,2,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,12.0,1
2,4,0,7.0,0,2.0,1.0,1,24.952712,0,1,0,768,1,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,2.0,5.0,2,1,0,33,3,42,0,0,7,5,3,35.0,5,0,1,0,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,11.0,1
3,4,0,7.0,0,2.0,1.0,1,24.952712,0,1,0,768,3,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,2.0,5.0,2,0,0,33,3,37,0,5,7,5,8,26.0,3,0,1,0,0.000011,0.0,7,0.000000,1,0.000000e+00,0.0,0,0,0.0,0.0,10.0,1
4,4,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,768,3,4,1.0,0,1,0.003175,13,1.0,1.0,8,1,4.0,5.0,2,0,1,33,3,42,0,5,7,5,7,29.0,5,0,1,2,0.000011,0.0,7,0.000281,1,4.290840e-07,0.0,0,0,0.0,1.0,3.0,1


# 3. Detect Covariate Shift: Detect Dataset Shift feature : Skip for this experiment 1


# 4. Build classifier prediction without features having a drift value greater

In [0]:

from sklearn.model_selection import train_test_split
#Get the values and run a classifier 
features= listAcceptField_Test#.remove('HasDetections')

# convert to dummies
X=df_train[features].copy()
y=df_train['HasDetections']

# 70% train, 30% test from dataset train.csv
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

X_train

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
918219,4,0,7.0,0,1.0,1.0,1,24.876511,0,1,0,256,2,6,1.0,0,0,0.0,9,1.0,1.0,3,1,4.0,5.0,2,1,0,17,6,22,0,2,12,13,3,8.0,0,0,1,0,0.000006,0.0,3,0.000271,1,0.0,0.0,1,1,1.0,0.0,3.0
975645,4,0,7.0,0,1.0,1.0,1,24.876511,0,2,0,256,1,6,1.0,0,0,0.0,9,1.0,1.0,2,1,2.0,5.0,2,3,0,12,1,22,2,0,12,13,7,8.0,3,0,1,2,0.000006,0.0,3,0.000271,0,0.0,0.0,0,0,0.0,0.0,3.0
495842,4,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,256,4,6,1.0,0,0,0.0,9,1.0,1.0,2,1,8.0,5.0,2,3,0,12,1,22,0,6,12,13,6,8.0,2,0,1,2,0.000006,0.0,3,0.000000,0,0.0,0.0,0,0,0.0,0.0,15.0
357464,4,0,7.0,0,1.0,1.0,1,27.000000,0,1,0,768,6,4,1.0,0,0,0.0,9,1.0,1.0,7,1,4.0,5.0,2,0,0,24,3,22,0,16,2,2,5,26.0,5,0,1,2,0.000006,0.0,3,0.000000,0,0.0,0.0,0,0,0.0,0.0,10.0
876131,4,0,7.0,0,3.0,1.0,1,27.000000,0,1,0,256,3,6,1.0,0,0,0.0,3,1.0,1.0,2,1,4.0,1.0,2,0,0,12,1,22,0,4,12,13,8,24.0,3,0,1,2,0.000000,0.0,3,0.000000,0,0.0,0.0,0,0,0.0,1.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595096,4,0,7.0,0,1.0,1.0,1,24.876511,0,1,0,768,2,4,1.0,0,0,0.0,9,1.0,1.0,7,1,4.0,5.0,2,0,0,24,3,22,0,2,5,5,3,29.0,3,0,1,0,0.000006,0.0,4,0.000271,1,0.0,0.0,0,0,0.0,0.0,3.0
775929,4,0,7.0,0,1.0,1.0,1,27.000000,0,1,0,768,3,4,1.0,0,0,0.0,9,1.0,1.0,7,1,4.0,5.0,2,0,0,24,3,22,0,4,5,5,8,8.0,3,0,1,0,0.000006,0.0,3,0.000000,1,0.0,0.0,0,0,0.0,0.0,1.0
739640,4,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,768,1,4,1.0,0,0,0.0,9,1.0,1.0,7,1,4.0,1.0,2,1,0,24,3,22,0,0,2,2,2,37.0,5,0,1,2,0.000006,0.0,3,0.000271,0,0.0,0.0,0,0,0.0,1.0,7.0
979611,4,0,7.0,0,1.0,1.0,1,27.000000,0,1,0,768,4,4,1.0,0,0,0.0,9,1.0,1.0,1,1,4.0,5.0,2,0,0,11,3,22,0,6,5,5,2,10.0,2,0,1,0,0.000006,0.0,3,0.000271,0,0.0,0.0,1,1,0.0,0.0,10.0


In [0]:

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

# main Model
clf = ExtraTreesClassifier(n_estimators=1000, max_depth=4)
clf.fit(X_train,y_train)

# test with dummy model
#dummy_clf = DummyClassifier() # generates predictions by respecting the training set's class distribution
#dummy_clf.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=4, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [0]:
#scores = cross_val_score(clf, X_train, y_train, cv=10,scoring=make_scorer(accuracy_score))
#dummy_scores = cross_val_score(dummy_clf, X_train, y_train, cv=10, scoring=make_scorer(accuracy_score))

#Report the results of 10-Kfold stratified cross-validation of 2 method
#print("ACC: %0.2f +/- %0.2f" % (scores.mean(), scores.std()))
#print("Dummy ACC: %0.2f +/- %0.2f" % (dummy_scores.mean(), dummy_scores.std()))

# 4.1 Test On dataset test.csv and download result

In [0]:
# Load Test Data all
#df_test = pd.read_csv("test.csv",skiprows=0)
features= listAcceptField_Test
testFeatures=df_test[features].copy()
pred = clf.predict(testFeatures)
columns = ['HasDetections']
sub = pd.DataFrame(data=pred,columns=columns)
sub['MachineIdentifier'] = df_test_id
sub = sub[['MachineIdentifier','HasDetections']]
sub.to_csv('predict_without_drift_value_greater.csv', index=False)

In [0]:
#download result
#from google.colab import files
#files.download('predict_without_drift_value_greater.csv')

In [0]:
sub

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,1
1,00000176ac758d54827acd545b6315a5,1
2,0000019dcefc128c2d4387c1273dae1d,0
3,0000055553dc51b1295785415f1a224d,0
4,00000574cefffeca83ec8adf9285b2bf,1
...,...,...
7853248,fffff8c0e065c468a2373f7afd5e7674,1
7853249,fffff90b27a1248b6fffc7a535bd736c,1
7853250,fffffa6a956c17ddbabca53d4ab708ae,1
7853251,fffffad7b6c8196ec5cae634406c0d4f,0


## Upload File to Google Drive due to can not download via browser a big file

In [0]:
!zip data2.zip predict_without_drift_value_greater.csv

  adding: predict_without_drift_value_greater.csv (deflated 49%)


In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
upload = drive.CreateFile({'title': 'project_ex_1.zip'})
upload.SetContentFile('data2.zip')
upload.Upload()